# Language

In [23]:
import os
from pathlib import Path
import json
import pickle

import tqdm 
from icl.constants import DEVICE

DATA = Path('../data')

def gen_from_jsonl(file_path, num_lines=5_000_000, start=0):
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            if i == start:
                break

        for i, line in tqdm.tqdm(enumerate(file), total=num_lines, desc=f'Loading {file_path}'):
            if i >= start + num_lines:
                break

            yield json.loads(line)


with open(DATA / 'bigram_freq_percents.pkl', 'rb') as file:
    bigram_freqs = pickle.load(file)

bigram_freqs.shape # token 1, token 2

(5000, 5000)

In [9]:
from transformer_lens import HookedTransformerConfig, HookedTransformer

model_cfg = HookedTransformerConfig(
    n_layers=2,
    d_model=256,
    d_head=32,
    n_heads=8,
    n_ctx=1024,
    d_vocab=5000,
    tokenizer_name='georgeyw/TinyStories-tokenizer-5k',
    normalization_type='LN',
    attn_only=True,
    seed=1,
    positional_embedding_type='shortformer',
)

model_cfg_one_layer = HookedTransformerConfig(
    n_layers=1,
    d_model=256,
    d_head=32,
    n_heads=8,
    n_ctx=1024,
    d_vocab=5000,
    tokenizer_name='georgeyw/TinyStories-tokenizer-5k',
    normalization_type='LN',
    attn_only=True,
    seed=1,
    positional_embedding_type='shortformer',
)

model = HookedTransformer(model_cfg)
tokenizer = model.tokenizer

Using pad_token, but it is not set yet.


In [33]:
import yaml
import torch
from typing import List

def translate_int_to_str(token_ints: List[int]):
    t = torch.tensor([token_ints], device=DEVICE)
    return model.to_str_tokens(t)


def preview_trigrams_dict(trigrams_dict):
    print("----- Top 1000 trigrams -----")
    for k, v in list(trigrams_dict.items())[:1000]:
        trigram = translate_int_to_str(k)
        print(f'{trigram}: {v}')


def gen_trigrams(file_path, tokenizer):
    for row in gen_from_jsonl(file_path):
        contents = row['contents']
        tokens = tokenizer(contents)['input_ids']

        for i in range(len(tokens) - 2):
            yield tokens[i:i+3]
   

def get_top_k_trigrams(file_path, k=10, multiplier=2, padding=10, get_count=lambda _: 1, reverse_count=lambda _: 1, tokenizer=tokenizer):
    table = {}

    for trigram in gen_trigrams(file_path, tokenizer):
        key = tuple(trigram)
        table[key] = table.get(key, 0.) + get_count(trigram)
        
        if len(table) > multiplier * k:
            table = dict(sorted(table.items(), key=lambda x: x[1], reverse=True)[:k + padding])
            print("----- Top 1000 trigrams -----")
            preview_trigrams_dict(table)

    return  dict({k: reverse_count(v) for k, v in sorted(table.items(), key=lambda x: x[1], reverse=True)[:k].items()})

def divide_by_last_2_bigram(trigram):
    # print(translate_int_to_str(trigram), bigram_freqs[trigram[1], trigram[2]])
    return 1. / (5000 * bigram_freqs[trigram[1], trigram[2]])

top_10_000_trigrams = get_top_k_trigrams(
    DATA / 'train-5m.jsonl', 
    k=10_000, 
    multiplier=100, 
    padding=10_000, 
    tokenizer=tokenizer, 
    get_count=divide_by_last_2_bigram
)

Loading ../data/train-5m.jsonl:   0%|          | 4255/5000000 [00:06<2:12:48, 626.91it/s]

----- Top 1000 trigrams -----
----- Top 1000 trigrams -----
['as', ' ', 'ario']: 12138.64272
['as', 'th', 'aking']: 9631.2784
['ard', '.', 'ara']: 7772.098533333333
[' ', ' ', ' heat']: 6388.75932631579
['\n', ' ', ' brand']: 6069.32136
[' ', ' ', ' couple']: 5921.289131707317
['V', ' ', ' movie']: 4668.708738461539
['\n', ' ', ' score']: 4668.708738461539
['the', ' ', 'ancy']: 4495.7936
[' ', ' ', ' increasing']: 4046.2142400000002
[' ', ' ', ' projects']: 3915.6912
[' ', ' ', ' nearly']: 3678.376581818182
[' ', ' ', ' miss']: 3371.8452
[' ', ' ', ' paper']: 3280.714248648649
[' ', ' ', ' opportunity']: 3236.9713920000004
[' ', ' ', ' max']: 3222.6485097345135
['ide', ',', '�']: 3050.7619333333337
[' in', '\n', ' others']: 3046.894975
['ow', '\n', ' cat']: 3046.894975
[' was', ' ', ' supposed']: 3034.66068
[' ', ' ', ' weeks']: 2924.9741493975907
['\n', ' ', ' yes']: 2758.782436363637
[' ', ' ', ' mass']: 2758.782436363637
[' ', 'the', 'ese']: 2745.2198769230768
['ce', 'an', 'wards']:

Loading ../data/train-5m.jsonl:   0%|          | 4395/5000000 [00:08<9:17:33, 149.33it/s] 

['is', 'b', 'oid']: 135.43248157894735
['R', 'ex', 'iter']: 135.2016
['ions', ' ', 'ared']: 135.17419510022273
['ar', 'p', 'nes']: 135.0883090909091
['ering', ' on', ' claimed']: 135.08310666666665
[',', ' saw', ' expected']: 135.0542
['.', ' g', 'ae']: 134.95096451612903
['r', 'atic', ' largest']: 134.9232
['iz', 'en', 'pri']: 134.8579692307692
[' R', ' M', ' Re']: 134.81731612903226
['H', 'oc', ' 2012']: 134.782
[' support', '.', ' prof']: 134.77627514450867
[' sw', 'im', ' spell']: 134.4887142857143
[',', ' —', 'Up']: 134.4603
['k', 'ur', 'ides']: 134.39634545454544
[' n', 'orth', ' They']: 134.39600000000002
['r', 'du', ' policies']: 134.262
['r', 'du', ' movement']: 134.262
['V', 'ot', ' iss']: 134.03013333333334
[' stat', 'u', 'ory']: 133.98426
[' G', 'P', ' whe']: 133.7932857142857
['74', 'b', ' 32']: 133.67361818181817
['\n', 'In', ' restrict']: 133.65480000000002


Loading ../data/train-5m.jsonl:   0%|          | 8401/5000000 [00:14<1:57:20, 709.03it/s]

...

[' ', 'the', 'et']: 626.1027789473684
['\n', ' ', ' very']: 625.7032329896907
[' ', ' ', ' ref']: 625.7032329896907
[' ', ' ', ' son']: 625.7032329896907
['\n', ' ', ' front']: 625.7032329896907
['w', 'ast', 'ocol']: 623.5982
['l', 'ast', ' valid']: 623.5982
[' ', ' ', ' Ass']: 623.1336098562629
['i', ' It', 'ya']: 622.576
['\n', ' ', ' came']: 622.4944984615385
['ion', ' from', 'eth']: 619.94155
['ERS', ' v', 'ER']: 619.5292000000001
['gy', ' v', 'uth']: 619.5292000000001
[' "', 'he', 'As']: 619.1162
['ft', 'he', 'IS']: 619.1162
['man', ' D', 'iday']: 618.648
['l', ',', '.,']: 617.0080314606741
['.', ' ', ' Le']: 616.8457280580761
[' The', ' ', ' line']: 616.1747573604061
['\n', ' ', ' letter']: 616.1747573604061
[').', ' ', ' Secretary']: 616.1747573604061
[' ', ' ', ' Secretary']: 616.1747573604061
[' ', ' ', ' line']: 616.1747573604061
['1', ';', 'ful']: 616.0852
[' ', 'the', 'og']: 615.3079034482759
['\n', ' ', ' written']: 613.0627636363637
['e', '-', ' bi']: 612.8601647058823
['

Loading ../data/train-5m.jsonl:   1%|          | 29395/5000000 [00:52<2:03:12, 672.34it/s] 

----- Top 1000 trigrams -----
----- Top 1000 trigrams -----
['am', ',', 'isk']: 27456.8574
['\n', '\n', '�']: 24375.1598
['a', ',', 'ared']: 13728.4287
['as', ' ', 'ario']: 12138.64272
[' ', ' ', ' spokes']: 12138.64272
[' T', 'the', 'ior']: 11895.952800000001
['the', ' in', 'uth']: 11283.028400000001
['as', 'th', 'aking']: 9631.2784
['\n', ' ', ' fan']: 9337.417476923078
[' ', ' ', ' slight']: 9337.417476923078
[' Rich', '\n', 'ird']: 8125.053266666668
[' ', '\n', ' follow']: 8125.053266666668
['cer', ' of', 'ry']: 7958.930700000001
['ard', '.', 'ara']: 7772.098533333333
[' of', ' ', 'adow']: 7140.378070588235
[' ', ' ', ' mov']: 6949.604610687023
['i', ',', 'cher']: 6864.21435
['ia', ',', 'vil']: 6864.21435
[' ', ' ', ' heat']: 6388.75932631579
[')', ' ', ' couldn']: 6388.75932631579
['.', ' ', ' Democr']: 6388.75932631579
['ail', '\n', 'oad']: 6093.78995
[',', '\n', ' introdu']: 6093.78995
['\n', ' ', ' brand']: 6069.32136
['port', ' ', ' offers']: 6069.32136
['\n', ' ', ' saf']: 60

Loading ../data/train-5m.jsonl:   1%|          | 32019/5000000 [00:58<2:31:57, 544.86it/s] 


KeyboardInterrupt: 

TODOs
- Normal top-k trigrams
- Normal top-k skip trigrams
- Adjusted (by last-2 bigram freq) top-k trigrams
- Adjusted (by last-2 bigram freq) top-k skip trigrams

Partition into subsets of 100k rows:
- Repeat for each. Look at differences/commonalities. 
